In [1]:
import pandas as pd

In [2]:
train_ds = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/persuade/merged_train_ds.csv')

In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, AutoConfig

num_labels = 2

import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
device

device(type='cuda', index=0)

In [10]:
DOWNLOAD_MODEL = False
MODEL_NAME = 'distilroberta-base'
MODEL_DIR = '/home/peng_sun2/s3shared/kaggle/llm-2023/distilroberta'

if DOWNLOAD_MODEL:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME);
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    config = AutoConfig.from_pretrained(MODEL_DIR)
    config.num_labels= num_labels
    model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path = MODEL_DIR, config = config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /home/peng_sun2/s3shared/kaggle/llm-2023/distilroberta and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

num_train_epochs=16.0


In [12]:
metric_name = "roc_auc"
model_name = "distilroberta"#"deberta-large"
batch_size = 2

args = TrainingArguments(
    f"{model_name}-finetuned_v5",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    lr_scheduler_type = "cosine",
    
    optim="adamw_torch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=8,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to='none',
    save_total_limit=2,
    
)

In [16]:
train_ds.head(3)

,prompt_name,prompt,source,text,generated
0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Cars. Cars have been around since they became ...,0
1,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...",Transportation is a large necessity in most co...,0
2,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ...","""America's love affair with it's vehicles seem...",0


In [27]:
from sklearn.model_selection import StratifiedGroupKFold

cv = StratifiedGroupKFold(n_splits=5,shuffle=True,random_state=42)

for idx, (train_idx, val_idx) in enumerate(cv.split(
        train_ds[['prompt_name', 'prompt', 'source', 'text']], 
        train_ds['generated'], 
        groups = train_ds['prompt_name'])):
#     train_x, train_y = train_ds.loc[train_idx, ['prompt_name', 'prompt', 'source', 'text']],\
#         train_ds.loc[train_idx, ['generated']]
#     val_x, val_y = train_ds.loc[val_idx, ['prompt_name', 'prompt', 'source', 'text']],\
#         train_ds.loc[val_idx, ['generated']]
    train_split = train_ds.loc[train_idx, ['prompt_name', 'prompt', 'source', 'text', 'generated']]
    val_split = train_ds.loc[val_idx, ['prompt_name', 'prompt', 'source', 'text', 'generated']]
    break;

/home/peng_sun2/.local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:909: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [28]:
from datasets import Dataset
ds_train = Dataset.from_pandas(train_split)
ds_valid = Dataset.from_pandas(val_split)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyarrow/pandas_compat.py:358: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [30]:
train_split.generated.value_counts()

generated
0    12364
1        3
Name: count, dtype: int64

In [31]:
val_split.generated.value_counts()

generated
0    1886
Name: count, dtype: int64